In [1]:
from copy import deepcopy
from genericpath import isdir
import sys
sys.path.append('/home/insign/Doc/insign/Mask_yolo')
import torch
import cv2
import numpy as np 
from model.od.data.datasets import letterbox
from typing import Any
from model.backbone_YOLO import *
from model.head_RCNN import *
from model.groundtrue_import import *
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import pydensecrf.densecrf as dcrf

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [3]:
cfgPath = '/home/insign/Doc/insign/Mask_yolo/config/yolocfg.yaml'
roimodelPath = '/home/insign/Doc/insign/Mask_yolo/run/0824_fcnhead_size640/best.pt'

#   Loading config file
with open(cfgPath, 'r') as f:
        cfg = yaml.load(f, Loader=yaml.Loader)

#   Setting device
device = 'cpu'

#   Model yolo_backbone loading
yolo = model_manipulate(cfg['model']['weight']).eval().to(device)

#   Model roi_head loading
mask_head = torch.load(roimodelPath).eval().to(device)

In [4]:
imagePath = '/home/insign/Doc/insign/Mask_yolo/Polyp/Validate/44.jpg'

def image_loading(img_path):
    image = cv2.imread(str(img_path))
    img_h, img_w = image.shape[0], image.shape[1]
    image = letterbox(image, new_shape=480)[0]
    im0s = deepcopy(image)
    image = image[:, :, ::-1].transpose(2, 0, 1)
    image = np.ascontiguousarray(image)
    image = torch.from_numpy(image).unsqueeze(0).to(device)
    image = image.float()
    image /= 255.0
    return image, im0s, img_h, img_w

image, im0s, img_h, img_w = image_loading(imagePath)

In [5]:
def model_detection(image, yolo, mask_head, cfg):
    #   Predict bounding box
    pred = yolo(image)
    rois = non_max_suppression(pred['rois'][0],cfg['nms']['conf_thres'], cfg['nms']['iou_thres'], classes= cfg['nms']['classes'],agnostic=cfg['nms']['agnostic_nms'])
    boxes = rois[0][:,:4]#  rois
    if len(boxes) == 0:
        return [], []
    feature_map = featuremapPack(pred['feature_map']) #   extract feature map and boxes
    cv2.rectangle(im0s, (int(boxes[0][0]), int(boxes[0][1])), (int(boxes[0][2]), int(boxes[0][3])), (0, 255, 0), 2)

    #   Resize to bounding box
    h = int(boxes[0][2] - boxes[0][0])
    w = int(boxes[0][3] - boxes[0][1])
    resize_to_bbs = transforms.Compose([transforms.Resize((w, h))])
    
    #   Predict mask
    mask_logits = mask_head(feature_map, boxes)
    mask = mask_logits.detach().cpu().numpy()
    mask_logits = resize_to_bbs(mask_logits)
    mask_logits = mask_logits.detach().cpu().numpy()
    mask_logits = mask_logits[0][0]>0.5
    mask_logits.dtype = 'uint8'
    mask_logits = mask_logits*255
    mask_logits = cv2.cvtColor(mask_logits, cv2.COLOR_GRAY2RGB)
    return boxes, mask_logits, mask


In [6]:
#   prediction
boxes, mask_logits, mask = model_detection(image, yolo, mask_head, cfg)

mask = mask[0]
mask = mask.reshape((1,-1))

d = dcrf.DenseCRF2D(126, 126, 1)
d.addPairwiseGaussian(sxy=(10,10), compat=3, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
d.setUnaryEnergy(mask)

mask = mask.reshape((126,126))
mask = mask>0.5
mask.dtype = 'uint8'
mask = mask*255
mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
cv2.imwrite('mask.jpg', mask)

/usr/local/lib/python3.8/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


True